In [ ]:
\[media pointer="file-service://file-9hNEafhmnArXBXr9mJ6EZR"]
\[media pointer="file-service://file-8yBnJNT549FwEcVJTmjmy8"]
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from qiskit import QuantumCircuit, transpile
from qiskit\_aer import Aer
from qiskit\_aer.noise import NoiseModel, thermal\_relaxation\_error
from scipy.stats import entropy

# --- 실험 설정 ---

shots = 1024
steps = 30
initial\_n = 2
delay\_time = 30000  # ns
T1 = 50000
T2 = 30000
threshold = 5  # dominant 상태 반복 시 확장 조건

# --- 디코히어런스 노이즈 모델 구성 ---

noise\_model = NoiseModel()
sim = Aer.get\_backend("aer\_simulator")
sim.set\_options(method="density\_matrix")
for q in range(20):  # 여유 큐비트 수 설정
error = thermal\_relaxation\_error(T1, T2, delay\_time)
noise\_model.add\_quantum\_error(error, "delay", \[q])

# --- 회로 생성 함수: 얽힘 + 디코히어런스 포함 ---

def build\_entangled\_circuit(U):
n = len(U)
qc = QuantumCircuit(n, n)
for i, theta in enumerate(U):
qc.ry(theta, i)
qc.delay(delay\_time, i, unit='ns')
for i in range(n - 1):  # 얽힘 삽입 (인접 CX)
qc.cx(i, i + 1)
qc.barrier()
qc.measure(range(n), range(n))
return qc

# --- 실험 루프 ---

U = \[np.random.uniform(0, np.pi) for \_ in range(initial\_n)]
dominant\_streak = 0
last\_dominant = None
n\_history = \[]
entropy\_trajectory = \[]

for step in range(steps):
qc = build\_entangled\_circuit(U)
qc = transpile(qc, sim, optimization\_level=0)
result = sim.run(qc, shots=shots, noise\_model=noise\_model).result()
counts = result.get\_counts()

```
# 엔트로피 계산
probs = np.array(list(counts.values())) / shots
entropy_trajectory.append(entropy(probs, base=2))

# dominant 상태 판단
dominant = max(counts.items(), key=lambda x: x[1])[0]
if dominant == last_dominant:
    dominant_streak += 1
else:
    dominant_streak = 1
    last_dominant = dominant

# 구조 확장 조건 충족 시
if dominant_streak >= threshold:
    U.append(np.pi / 4)  # 새 큐비트 추가
    dominant_streak = 0

n_history.append(len(U))
```

# --- 시각화 ---

plt.figure(figsize=(10, 4))
plt.plot(n\_history, marker='o')
plt.title("Qubit Count Over Time (Entanglement + Structural Expansion)")
plt.xlabel("Step")
plt.ylabel("Qubit Count")
plt.grid(True)
plt.tight\_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(entropy\_trajectory, marker='o')
plt.title("Entropy Over Feedback Steps (Dual-Mechanism Structure)")
plt.xlabel("Step")
plt.ylabel("Entropy (bits)")
plt.grid(True)
plt.tight\_layout()
plt.show()


![Model A result](../01_papers/figures/result_1.png)